In [31]:
# Import Libraries
import shutil
import pandas as pd
import os
from pptx import Presentation
from pptx.chart.data import CategoryChartData
from pptx.util import Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
import numpy as np 
from datetime import datetime, timedelta
import random
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN

# Load the power point presentation
ppt = Presentation(r"/Users/da/Desktop/Files/Data Science & Analytics/Python Projects/Update PowerPoint Project/Report_Template.pptx")

In [32]:
# The original script was querying the data using SQL, but in order for this script to work for anyone running it, 
# the code will generate random values

# Number of rows 
n =  random.randint(4000,5000)

# Generate case numbers
case_number = np.random.choice(range(10000,50000), size = n, replace = False)

# ------------------------------- Create Data for Current Year ---------------------------------------
# Create dates 
today = datetime.today()
start_of_year = datetime(today.year,1,1)
first_of_this_month = datetime(today.year, today.month,1)

end_of_last_month = first_of_this_month - timedelta(days=1)
start_of_last_month = datetime(end_of_last_month.year,end_of_last_month.month,1)

end_of_prior_month = start_of_last_month - timedelta(days=1)
start_of_prior_month = datetime(end_of_prior_month.year,end_of_prior_month.month,1)

total_days = (end_of_last_month - start_of_year).days + 1

# Generate dates dynamically from the beginning of the year until end of last month
case_received_date = [
    start_of_year + timedelta(days = np.random.randint(0,total_days))
    for _ in range(n)
]

# Generate patient enrollment type
enroll_type = np.random.choice(['New Patients','Existing Patients'], size=n)

# Create list of products 
products = ["Product A", "Product B", "Product C", "Product D","Product E","Product F", "Product G"]

# Create the dataframe 
enrollments_current_year = pd.DataFrame ({
    'CaseNumber' : case_number,
    'case_received_date' : case_received_date,
    'enrollment_type' : enroll_type})
enrollments_current_year["Product"] = np.random.choice(products, size = len(enrollments_current_year))

# ------------------------------- Create Data for Previous Year ---------------------------------------
# Number of rows
n2 =  random.randint(3900,4900)

# Generate case numbers
case_number_last_year = np.random.choice(range(10000,50000), size = n2, replace = False)

# Create dates
start_of_last_year = datetime(today.year - 1,1,1)
first_of_last_month_last_year = datetime(today.year - 1, today.month, 1)
end_of_last_month_last_year = first_of_last_month_last_year - timedelta(days=1)
total_days_last_year = (end_of_last_month_last_year - start_of_last_year).days + 1

# Generate dates dynamically for the previous year
case_received_date_last_year = [
    start_of_last_year + timedelta(days = np.random.randint(0,total_days))
    for _ in range(n2)
]

# Generate patient enrollment type 
enroll_type_last_year = np.random.choice(['New Patients','Existing Patients'], size = n2)

# Create the dataframe
enrollments_last_year = pd.DataFrame({
    'CaseNumber' : case_number_last_year,
    'case_received_date' : case_received_date_last_year,
    'enrollment_type': enroll_type_last_year
})
enrollments_last_year['Product'] = np.random.choice(products, size = len(enrollments_last_year))

# ------------------------------ Create Rejection Data -----------------------------------------------
# Number of rows
n3 = 9

# Generate application number
application_number = np.random.choice(range(49000,50000), size = n3, replace = False)

# Generate dates dynamically for application date within the last month
total_days_app = (end_of_last_month - start_of_last_month).days - 5

application_received_date = [
    start_of_last_month + timedelta(days = np.random.randint(0,total_days_app))
    for _ in range(n3)
]

# Generate dates dynamically for determination date within the last month that it within 5 days after the application  date
determination_date = [
    app_date + timedelta(days = np.random.randint(1,6))
    for app_date in application_received_date
]

# Creaet first and last name for patients
names = random.sample([
    "Olivia Johnson", "Liam Smith", "Emma Brown", "Noah Davis",
    "Ava Miller", "Ethan Wilson", "Sophia Moore", "Mason Taylor",
    "Isabella Anderson", "Lucas Thomas", "Mia White", "James Harris"
],n3)

# Create list of rejection reasons
reject_reasons = [
    "Missing required medical documentation",
    "Drug not covered under insurance plan",
    "Patient does not meet eligibility criteria",
    "Prior authorization denied",
    "Invalid patient information",
    "Application submitted past deadline"
]

# Create the dataframe
rejection_data = pd.DataFrame({
    'Application_Number' : application_number,
    'Application_Date': application_received_date,
    'Determination_Date': determination_date,
    'Patient_Name' : names
})
rejection_data['Product'] = np.random.choice(products, size = len(rejection_data))
rejection_data['Rejection_Reason'] = np.random.choice(reject_reasons, size = len(rejection_data))

print(rejection_data)


   Application_Number Application_Date Determination_Date       Patient_Name  \
0               49934       2025-07-08         2025-07-09         Emma Brown   
1               49268       2025-07-09         2025-07-14       Ethan Wilson   
2               49713       2025-07-18         2025-07-23         Ava Miller   
3               49787       2025-07-10         2025-07-14       Lucas Thomas   
4               49162       2025-07-09         2025-07-13  Isabella Anderson   
5               49165       2025-07-11         2025-07-15     Olivia Johnson   
6               49679       2025-07-02         2025-07-03         Noah Davis   
7               49476       2025-07-22         2025-07-25       Mason Taylor   
8               49153       2025-07-20         2025-07-21       James Harris   

     Product                            Rejection_Reason  
0  Product G                 Invalid patient information  
1  Product A       Drug not covered under insurance plan  
2  Product F          

In [33]:
# Clean and format the data for the first slide 

# Create Month_Period for grouping
enrollments_current_year['Month_Period'] = enrollments_current_year['case_received_date'].dt.to_period('M')
enrollments_last_year['Month_Period'] = enrollments_last_year['case_received_date'].dt.to_period('M')

# Create function to calculate enrollments by month 
def create_enrollment_pivot(enrollments, date_col='case_received_date', id_col='CaseNumber', type_col='enrollment_type'):

    # Group by enrollment_type and Month_Period
    enrollments_grouped = (
        enrollments.groupby([type_col, 'Month_Period'])
        .agg(Enroll_Count=(id_col, 'count'))
        .reset_index()
    )

    # Sort by Month_Period
    enrollments_grouped = enrollments_grouped.sort_values('Month_Period')

    # Create display column
    enrollments_grouped['Month'] = enrollments_grouped['Month_Period'].dt.strftime('%b-%y')

    # Remove Month_Period if not needed
    enrollments_grouped = enrollments_grouped.drop(columns=['Month_Period'])

    # Get month order for proper sorting in pivot
    month_order = enrollments_grouped['Month'].drop_duplicates().tolist()

    # Create pivot table
    enrollments_pivoted = (
        enrollments_grouped.pivot_table(
            index='Month',
            columns=type_col,
            values='Enroll_Count',
            aggfunc='sum',
            fill_value=0
        )
        .reindex(month_order)
    )

    return enrollments_pivoted.reset_index()


enrollments_current_year_clean = create_enrollment_pivot(enrollments_current_year)
enrollments_last_year_clean = create_enrollment_pivot(enrollments_last_year)


# --------------------- Calculate enrollment metrics -------------------------
# Calculate total enrollments_current_year for new patients in the previous month
total_enroll_new = enrollments_current_year[
    (enrollments_current_year['enrollment_type'] == 'New Patients') &
    (enrollments_current_year['case_received_date'].between(start_of_last_month,end_of_last_month))
]['CaseNumber'].count()

# Calculate total enrollments_current_year for existing patients in the previous month
total_enroll_ex = enrollments_current_year[
    (enrollments_current_year['enrollment_type'] == 'Existing Patients')&
    (enrollments_current_year['case_received_date'].between(start_of_last_month,end_of_last_month))
]['CaseNumber'].count()

# Calculate total enrollments_current_year for all patients in the previous month
total_enroll = enrollments_current_year[
    enrollments_current_year['case_received_date'].between(start_of_last_month,end_of_last_month)
]['CaseNumber'].count()

# Calculate total enrollments_current_year for all patients in the prior month
total_enroll_prior = enrollments_current_year[
    enrollments_current_year['case_received_date'].between(start_of_prior_month,end_of_prior_month)
]['CaseNumber'].count()

# Calculate percentage change between prior month and last month
enroll_change = (total_enroll - total_enroll_prior) / total_enroll_prior
enroll_change = f'{int(round(enroll_change*100))}%'

# Calculate average monthly enrollments_current_year
avg_enroll = int(round(
    enrollments_current_year.groupby('Month_Period')['CaseNumber']
    .count()
    .mean()
    ,0
))

# Calculate the product with the highest & lowest enrollment counts
product_count = enrollments_current_year.groupby('Product')['CaseNumber'].count()
highest_product = product_count.idxmax()
lowest_product = product_count.idxmin()

print(enrollments_current_year_clean)


enrollment_type   Month  Existing Patients  New Patients
0                Jan-25                369           378
1                Feb-25                312           359
2                Mar-25                353           375
3                Apr-25                362           345
4                May-25                359           365
5                Jun-25                355           358
6                Jul-25                314           369


In [ ]:
# Access the first slide
slide1 = ppt.slides[0]

# Find the first chart shape
chart_shape_1 = [shape for shape in slide1.shapes if shape.has_chart][0]

# Create an empty CategoryChartData object
chart_shape_1_data = CategoryChartData()

# Set the x-axis labels for the chart using the 'Month' column
chart_shape_1_data.categories = enrollments_current_year_clean["Month"].tolist()

# Add a data series for each enrollment type (all columns except 'Month')
for et in enrollments_current_year_clean.columns.drop("Month"):
    chart_shape_1_data.add_series(et, enrollments_current_year_clean[et].tolist())

# Replace chart data
chart_shape_1.chart.replace_data(chart_shape_1_data)

# Dictionary of shapes and their corresponding values
box_updates = {
    'New Patients' : total_enroll_new,
    'Existing Patients' : total_enroll_ex,
    'Percentage Prior Month' : enroll_change,
    'Average Monthly Enroll' : avg_enroll,
    'Highest Product' : highest_product,
    'Lowest Product' : lowest_product
}

# Update box values
for shape in slide1.shapes:
    if shape.has_text_frame and shape.name in box_updates:
        shape.text_frame.text = str(box_updates[shape.name])
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.name = 'Calibri'
                run.font.size = Pt(20)



In [35]:
# Clean and format the data for the second slide 

# Create only a month column to join the dataframes
enrollments_current_year_clean['Month_Name'] = pd.to_datetime(
    enrollments_current_year_clean['Month'], format='%b-%y'
).dt.strftime('%B')

enrollments_last_year_clean['Month_Name'] = pd.to_datetime(
    enrollments_last_year_clean['Month'], format = '%b-%y'
).dt.strftime('%B')

# Separate the dataframe for New and Existing Patients
enroll_new_current_year = enrollments_current_year_clean[['Month_Name','New Patients']].rename(columns = {'New Patients': 'New Patients CY'})
enroll_ex_current_year = enrollments_current_year_clean[['Month_Name','Existing Patients']].rename(columns = {'Existing Patients': 'Existing Patients CY'})

enroll_new_last_year = enrollments_last_year_clean[['Month_Name','New Patients']].rename(columns = {'New Patients':'New Patients PY'})
enroll_ex_last_year = enrollments_last_year_clean[['Month_Name','Existing Patients']].rename(columns = {'Existing Patients':'Existing Patients PY'})

# Join Current Year data with Prior Year Data
enroll_new_cy_py = pd.merge(enroll_new_current_year, enroll_new_last_year, on = 'Month_Name', how = 'inner')
enroll_ex_cy_py = pd.merge(enroll_ex_current_year, enroll_ex_last_year, on = 'Month_Name', how = 'inner')

enroll_new_cy_py = enroll_new_cy_py.set_index('Month_Name')
enroll_ex_cy_py = enroll_ex_cy_py.set_index('Month_Name')

# Calculate percentage change between CY and PY for last month
def percentage_change(enrollments,CY,PY):
    new_df = enrollments.copy()
    new_df['Month'] = pd.to_datetime(new_df.index + ' 2025', format='%B %Y')
    new_df = new_df.set_index('Month')
    latest_month = new_df.index.max()

    per_change_new = round(((new_df.loc[latest_month,CY] - new_df.loc[latest_month,PY])
                  / new_df.loc[latest_month,PY])*100)
    per_change_new = f"{per_change_new}%"
    return per_change_new

per_change_new_pt = percentage_change(enroll_new_cy_py,'New Patients CY','New Patients PY')
per_change_ex_pt = percentage_change(enroll_ex_cy_py,'Existing Patients CY','Existing Patients PY')



print(per_change_ex_pt)

7%


In [36]:
# Access the second slide
slide2 = ppt.slides[1]

# Find the charts
chart_shape_2 = [shape for shape in slide2.shapes if shape.has_chart][0]
chart_shape_3 = [shape for shape in slide2.shapes if shape.has_chart][1]

# Create an empty CategoryChartData object
chart_shape_2_data = CategoryChartData()
chart_shape_3_data = CategoryChartData()

# Take the row labels, convert them to strings and use them as the x-axis labels for the chart
chart_shape_2_data.categories = enroll_new_cy_py.index.astype(str).tolist()
chart_shape_3_data.categories = enroll_ex_cy_py.index.astype(str).tolist()

# Loop through each column to get the data and convert the values into a list
for et in enroll_new_cy_py.columns:
    chart_shape_2_data.add_series(et,enroll_new_cy_py[et].tolist())

for et in enroll_ex_cy_py.columns:
    chart_shape_3_data.add_series(et,enroll_ex_cy_py[et].tolist())

 # Replace chart data
chart_shape_2.chart.replace_data(chart_shape_2_data)
chart_shape_3.chart.replace_data(chart_shape_3_data)  

# Dictionary of shapes and their corresponding values
box_updates2 = {
    'New Patients Percentage Change' : per_change_new_pt,
    'Ex Patients Percentage Change' : per_change_ex_pt
}

# Update box values 
for shape in slide2.shapes:
    if shape.has_text_frame and shape.name in box_updates2:
        shape.text_frame.text = str(box_updates2[shape.name])
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.name = 'Calibri'
                run.font.size = Pt(20)
                run.bold = True


In [37]:
# Calculate metrics
# Total Rejections
total_rejections = rejection_data['Application_Number'].count()

# Top Rejected Reason
reason_counts = rejection_data.groupby('Rejection_Reason')['Application_Number'].count()
top_rejected_reason = reason_counts.idxmax()
top_rejected_count = reason_counts.max()

# Average time to rejection decision
rejection_data_copy = rejection_data.copy()
rejection_data_copy['time_diff_days'] = (rejection_data_copy['Determination_Date'] - rejection_data_copy['Application_Date']).dt.days
avg_time = round(rejection_data_copy['time_diff_days'].mean(),1)

#print(avg_time)

# Access the third slide
slide3 = ppt.slides[2]

# Find the table shape
table = None
for shape in slide3.shapes:
    if shape.has_table:
        table_shape = shape
        table = shape.table
        break

# Fill in the table
for i in range(len(rejection_data)):
    table.cell(i + 1, 0).text = str(rejection_data.iloc[i, 0])  # Application Number
    table.cell(i + 1, 1).text = rejection_data.iloc[i, 1].strftime("%Y-%m-%d")  # Application Date
    table.cell(i + 1, 2).text = rejection_data.iloc[i, 2].strftime("%Y-%m-%d")  # Determination Date
    table.cell(i + 1, 3).text = rejection_data.iloc[i, 3]  # Patient Name
    table.cell(i + 1, 4).text = rejection_data.iloc[i, 4]  # Product Name
    table.cell(i + 1, 5).text = rejection_data.iloc[i, 5]  # Rejection Reason

    #Format the text
    for col_idx in range(6):
        cell = table.cell(i+1,col_idx)
        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.name = 'Calibri'
                run.font.size = Pt(12)

table_shape.left = Inches(0.9)
table_shape.top = Inches(1.5)

# Fill out the summary box 
text_box = None
for shape in slide3.shapes:
    if shape.name == 'TextBox Rejections':
        text_box = shape
        break
text_box.text = f'Total applications rejected this month: {total_rejections}\nTop rejection reason: {top_rejected_reason} ({top_rejected_count})\nAverage time to rejection decision: {avg_time} days'

for paragraph in text_box.text_frame.paragraphs:
    for run in paragraph.runs:
        run.font.name = 'Calibri'
        run.font.size = Pt(18)

for p in text_box.text_frame.paragraphs:
    p.alignment = PP_ALIGN.CENTER

# Fill out the title 
month_year_name = start_of_last_month.strftime('%B %Y')

text_box_1 = None
for shape in slide3.shapes:
    if shape.name == 'Title Box':
        text_box_1 = shape
        break
text_box_1.text = f'Applications Rejected in {month_year_name}'

for paragraph in text_box_1.text_frame.paragraphs:
    for run in paragraph.runs:
        run.font.name = 'Calibri'
        run.font.size = Pt(20)
        run.font.bold = True


In [38]:
# Save the powerpoint with all the changes
ppt.save(r"/Users/da/Desktop/Files/Data Science & Analytics/Python Projects/Update PowerPoint Project/Report_Template_Updated.pptx")
